

# Preprocessing & Data Visualisation

### Data Importation & fields selection

In [ ]:
!pip install Tashaphyne
!pip install Unidecode
!pip install aiogoogletrans
!pip install cltk

from datetime import datetime
from google.colab import files
from datetime import datetime
from google.colab import drive

import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from tashaphyne.stemming import ArabicLightStemmer
from __future__ import unicode_literals
from unidecode import unidecode
from textblob import TextBlob
from aiogoogletrans import Translator
import asyncio
import unicodedata

from cltk.corpus.utils.importer import CorpusImporter
from cltk.corpus.arabic.alphabet import *
from cltk.stop.arabic.stopword_filter import stopwords_filter as ar_stop_filter
from cltk.tokenize.word import WordTokenizer



import pandas as pd
import numpy as np
import pickle
import time
import json
import glob
import re
import os

import warnings
warnings.simplefilter("ignore")
nltk.download('stopwords')
nltk.download('punkt')
drive.mount('/content/drive')

     |████████████████████████████████| 245kB 6.6MB/s 
     |████████████████████████████████| 112kB 19.8MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=452777027309d18d9c8ee7796c235418edb43510ab3bd637e6517ff61d5f233e
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
     |████████████████████████████████| 245kB 6.1MB/s 
     |████████████████████████████████| 1.3MB 6.8MB/s 
     |████████████████████████████████| 143kB 16.6MB/s 
     |████████████████████████████████| 296kB 18.2MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=f446d6579a2813716ced9b54545579e81167ea4e4a9526baf83bd4243ffb0b76
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
     |████████████████████████████████| 634kB 6.7MB/s 
     |█████████████████████████

### Discover Our Dataset

In [ ]:
#Stats about Text
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

# train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))
# train['char_count'] = train['text'].str.len() ## this also includes spaces
# train['avg_char_per_word'] = train['text'].apply(lambda x: avg_word(x))
# stop = stopwords.words('arabic')
# train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
# train['emoji_count'] = train['text'].apply(lambda x: emoji_counter(x))
# train = train.sort_values(by='word_count',ascending=[0])
# train.head()

### TEXT CLEANING

#### Text standarisation

In [ ]:
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}
ArListem = ArabicLightStemmer()


def stem(text):
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)

import pyarabic.araby as araby
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)
    
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


#### Texts from Social Media

#### Deal with Hashtags in a string

In [ ]:
def split_hashtag_to_words(tag):
    tag = tag.replace('#','')
    tags = tag.split('_')
    if len(tags) > 1 :
        
        return tags
    pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")
    return pattern.findall(tag)

def clean_hashtag(text):
    words = text.split()
    text = list()
    for word in words:
        if is_hashtag(word):
            text.extend(extract_hashtag(word))
        else:
            text.append(word)
    return " ".join(text)
def is_hashtag(word):
    if word.startswith("#"):
        return True
    else:
        return False
def extract_hashtag(text):
    
    hash_list = ([re.sub(r"(\W+)$", "", i) for i in text.split() if i.startswith("#")])
    word_list = []
    for word in hash_list :
        word_list.extend(split_hashtag_to_words(word))
    return word_list


#### Dealing with emojis in a string

In [ ]:
with open('/content/drive/MyDrive/Emojis/emojis.csv','r',encoding='utf-8') as f:
    lines = f.readlines()
    emojis_ar = {}
    for line in lines:
        line = line.strip('\n').split(';')
        emojis_ar.update({line[0].strip():line[1].strip()})

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

def emoji_native_translation(text):
    text = text.lower()
    loves = ["<3", "♥",'❤']
    smilefaces = []
    sadfaces = []
    neutralfaces = []

    eyes = ["8",":","=",";"]
    nose = ["'","`","-",r"\\"]
    for e in eyes:
        for n in nose:
            for s in ["\)", "d", "]", "}","p"]:
                smilefaces.append(e+n+s)
                smilefaces.append(e+s)
            for s in ["\(", "\[", "{"]:
                sadfaces.append(e+n+s)
                sadfaces.append(e+s)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(e+n+s)
                neutralfaces.append(e+s)
            #reversed
            for s in ["\(", "\[", "{"]:
                smilefaces.append(s+n+e)
                smilefaces.append(s+e)
            for s in ["\)", "\]", "}"]:
                sadfaces.append(s+n+e)
                sadfaces.append(s+e)
            for s in ["\|", "\/", r"\\"]:
                neutralfaces.append(s+n+e)
                neutralfaces.append(s+e)

    smilefaces = list(set(smilefaces))
    sadfaces = list(set(sadfaces))
    neutralfaces = list(set(neutralfaces))
    t = []
    for w in text.split():
        if w in loves:
            t.append("حب")
        elif w in smilefaces:
            t.append("مضحك")
        elif w in neutralfaces:
            t.append("عادي")
        elif w in sadfaces:
            t.append("محزن")
        else:
            t.append(w)
    newText = " ".join(t)
    return newText

def is_emoji(word):
    if word in emojis_ar:
        return True
    else:
        return False
    
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

translator = Translator()
loop = asyncio.get_event_loop()
def translate_emojis(words):
    word_list = list()
    words_to_translate = list()
    for word in words :
        t = emojis_ar.get(word.get('emoji'),None)
        if t is None:
            word.update({'translation':'عادي','translated':True})
            #words_to_translate.append('normal')
        else:
            word.update({'translated':False,'translation':t})
            words_to_translate.append(t.replace(':','').replace('_',' '))
        word_list.append(word)
    return word_list

def emoji_unicode_translation(text):
    text = add_space(text)
    words = text.split()
    text_list = list()
    emojis_list = list()
    c = 0
    for word in words:
        if is_emoji(word):
            emojis_list.append({'emoji':word,'emplacement':c})
        else:
            text_list.append(word)
        c+=1
    emojis_translated = translate_emojis(emojis_list)
    for em in emojis_translated:
        text_list.insert(em.get('emplacement'),em.get('translation'))
    text = " ".join(text_list)
    return text
    
def clean_emoji(text):
    # text = emoji_native_translation(text)
    # text = emoji_unicode_translation(text)
    return text

In [ ]:
def clean_tweet(text):

    if not isinstance(text, str) : 
      # print(text)
      text=str(text)
    text = re.sub('#\d+K\d+', ' ', text)  # years like 2K19
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('@[^\s]+',' ',text)
    text = clean_hashtag(text)
    text = clean_emoji(text)
    return text

In [ ]:
def clean_text(text):
    ## Clean for tweets
    text = clean_tweet(text)
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    ## Remove Emojis
    text = remove_emoji(text)
    ## Convert text to lowercases
    text = text.lower()
    ## Remove stop words
    text = remove_stop_words(text)
    ## Remove numbers
    text = re.sub("\d+", " ", text)
    ## Remove Tashkeel
    text = normalizeArabic(text)
    #text = re.sub('\W+', ' ', text)
    text = re.sub('[A-Za-z]+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    #Stemming
    #text = stem(text)
    return text

# train['text'] = train['text'].apply(lambda x:clean_text(x))

In [ ]:
clean_text("@galalroushdy @KajoTm ما انا متأكد من كده 😂😂😂	")

'انا متاكد كده'

In [ ]:
clean_text('RT @alhanoufj__: انا خايف لقى ثاني🎧💕 https://t...	')

'انا خايف لقي ثاني'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/tweetsDataset/tweets.csv',sep=',',names=['ID','Tweet','Class']) 

In [ ]:
c = CorpusImporter('arabic')
c.list_corpora
c.import_corpus('arabic_text_perseus')  # ~/cltk_data/arabic/text/arabic_text_perseus/
word_tokenizer = WordTokenizer('arabic')
import nltk.stem.arlstem
df['Tweet'] =df['Tweet'].apply(lambda x: clean_text(x))
df['Tweet'] =df['Tweet'].apply(lambda x: ' '.join(word_tokenizer.tokenize(x)))
df['Tweet'] =df['Tweet'].apply(lambda x: ' '.join(ar_stop_filter(x)))
df['Tweet'] =df['Tweet'].apply(lambda x: araby.strip_tashkeel(x))
df['Tweet'] =df['Tweet'].apply(lambda x: stem(x))

#  df = pd.DataFrame(np.random.randn(100, 2))

# text = 'اللُّغَةُ الْعَرَبِيَّةُ جَمِيلَةٌ.'
# word_tokenizer.tokenize(x)
# araby.strip_tashkeel(x)

df[].tail()


,ID,Tweet,Class
94827,389849012956057600,ونو وكد تمم تحد زود جنن ضبط بلس د حول هتف جمهر...,trust
94828,37337,,trust
94829,884428772866498560,ونو وكد سلف قطر ٪ جيش صرصر,trust
94830,716683864257781760,زين ل دقق نخب دخل هدف شغل ثون ونو وثق ونك شفت ...,trust
94831,449178499962908673,دون دحح لوم ونو وثق فكر,trust


In [ ]:
df_not_empty =df[df["Tweet"] != '']
df_not_empty.to_csv('/content/drive/MyDrive/beforSplitedDataset/tweets.csv', index=False)

# msk = np.random.rand(len(df)) < 0.8
# train = df[msk]
# test = df[~msk]


In [ ]:
print(len(test))
print(len(train))
train.head()

16004
64843


,ID,Tweet,Class
0,295820374774513667,ردجرز ونو سوء عبن شبب منح خبر شور عرف قمم لعب ربل,anger
1,353619287652581376,شفف ونو عون ونو جنن,anger
2,888346524714979329,وفق ونو عصب قسس نقض زجج كوم برر حيل كذب شعر حب...,anger
3,870974517983752193,كرر ونو سوء جدد جرب حلل خبر يتم لغغ خفض لسع,anger
6,886636321342619653,ونو عصب بسط حلو هنس شكل,anger


In [ ]:
train = df.sample(frac = 0.75) 
test = df.drop(part_75.index) 
  
train.to_csv('/content/drive/MyDrive/splitedDataset/train.csv', index=False)
test.to_csv('/content/drive/MyDrive/splitedDataset/test.csv', index=False)

62443    ونو زعل قلتل ضوق صدق مشش هسب تنم زعل طول عمر ك...
31420                              نوس ودك قول علش بسط لحن
40938                                  هزز درب حنن ونو غيب
26102                                  ونو خوف فتح ولس لهه
12283    قول للل نوم سوع نسس بيت لجج عشش قوم سلل خلو له...
                               ...                        
59711      ميس ونو حزز لعب حمد نور وصل خبر قوف زمل هدد ملد
51911          هدد رجل بلل ملل نزل سقف دخخ زرق صعد قول غرف
85084                  ونو كدد ونو رجج درب هلل سكر زوج ششش
94071    بسط فرق قرب ءسس لون شبب مشش حسس لقق معو عدو كر...
48030    سمم مير فوز لحس حدد جدد رءي شبب حلو حلل كفء وج...
Name: Tweet, Length: 60635, dtype: object

**bold text**

In [ ]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["Tweet"], inplace=True)
len(df['Tweet'])

80847

In [ ]:
df.to_csv('/content/drive/MyDrive/cleanedDataset/tweets.csv', index=False)

In [ ]:
print('Anger:',len(df[df["Class"] == 'anger']))
print('Trust:',len(df[df["Class"] == 'trust']))
print('Happiness:',len(df[df["Class"] == 'happiness']))
print('Sadness:',len(df[df["Class"] == 'sadness']))
print('Fear:',len(df[df["Class"] == 'fear']))
print('Anticipation:',len(df[df["Class"] == 'anticipation']))
print('Disgust:',len(df[df["Class"] == 'disgust']))
print('Surprise:',len(df[df["Class"] == 'surprise']))


Anger: 1161
Trust: 13416
Happiness: 22847
Sadness: 12937
Fear: 11214
Anticipation: 11806
Disgust: 775
Surprise: 6691


In [ ]:

c = CorpusImporter('arabic')
c.list_corpora
c.import_corpus('arabic_text_perseus')  # ~/cltk_data/arabic/text/arabic_text_perseus/
word_tokenizer = WordTokenizer('arabic')
text = 'يرقصـــــــــــون اللُّغَةُ الْعَرَبِيَّةُ جَمِيلَةٌ.  '
# strip_tatweel(text)
' '.join (word_tokenizer.tokenize(text))
# araby.strip_tashkeel(text)

NameError: ignored



```
DUMMY CODE NOT USED
```



In [ ]:
# df['Tweet'].apply(lambda x: emoji_counter(x))
# df['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# ### Rare words removal
# freq = pd.Series(' '.join(train['text']).split()).value_counts()[-50:]
# freq = list(freq.index)
# train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# ### Remove Numbers from text


# ### Discouver Data again after cleaning
# train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))
# train['char_count'] = train['text'].str.len() ## this also includes spaces
# train['avg_char_per_word'] = train['text'].apply(lambda x: avg_word(x))
# stop = stopwords.words('arabic')
# train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
# train['emoji_count'] = train['text'].apply(lambda x: emoji_counter(x))
# train = train.sort_values(by='word_count',ascending=[0])
# final = []
# for index, row in train.iterrows():
#     if len(row['text'].split()) > 3:
#         final.append([row['text'],row['sentiment']])
# df = pd.DataFrame(final)
# df.columns = ['text','sentiment']
# df.to_csv('final_nice.csv',index=False)